In [1]:
import os
os.chdir('E:tasawar\ML_Labs')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
#from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [5]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 24, 8)          │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 20)             │           580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [7]:
checkpoints =  r'E:tasawar\ML_Labs\venv\Lab_9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:tasawar\ML_Labs\venv\Lab_9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [10]:

# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [12]:
import os
path_dataset =r'E:tasawar\ML_Labs\venv\Lab_9'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_train.csv')
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_validation.csv')
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(r'E:tasawar\ML_Labs\venv\AEP_test.csv')
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler      = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

e:\ML_Labs\venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [13]:
time_steps=24
num_features=21

In [14]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 4.982484579086304 sec


In [15]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0737 - mae: 0.0737 - mape: 27.0147
Epoch 1: val_loss improved from inf to 0.01555, saving model to E:\ML_Labs\venv\Lab_9\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 89s 30ms/step - loss: 0.0737 - mae: 0.0737 - mape: 27.0899 - val_loss: 0.0156 - val_mae: 0.0156 - val_mape: 7.3428
Epoch 2/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0156 - mae: 0.0156 - mape: 65.8559
Epoch 2: val_loss improved from 0.01555 to 0.01224, saving model to E:\ML_Labs\venv\Lab_9\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 85s 31ms/step - loss: 0.0156 - mae: 0.0156 - mape: 65.8597 - val_loss: 0.0122 - val_mae: 0.0122 - val_mape: 6.0447
Epoch 3/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0134 - mae: 0.0134 - mape: 109.9877
Epoch 3: val_loss did not improve from 0.01224
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 129s 26ms/step - loss: 0.0134 - mae: 0.0134 - mape: 109.9683 - val_loss: 0.0135 - val_mae: 0.0135 - val_mape: 5.6196
Epoch 4/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0126 - mae: 0.0126 - mape: 8.7062
Epoch 4: val_loss improved from 0.01224 to 0.01088, saving model to E:\ML_Labs\venv\Lab_9\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 78s 29ms/step - loss: 0.0126 - mae: 0.0126 - mape: 8.7129 - val_loss: 0.0109 - val_mae: 0.0109 - val_mape: 4.8413
Epoch 5/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0120 - mae: 0.0120 - mape: 231.5440
Epoch 5: val_loss improved from 0.01088 to 0.00995, saving model to E:\ML_Labs\venv\Lab_9\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 86s 31ms/step - loss: 0.0120 - mae: 0.0120 - mape: 231.5884 - val_loss: 0.0099 - val_mae: 0.0099 - val_mape: 4.1877
Epoch 6/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0115 - mae: 0.0115 - mape: 28.4319
Epoch 6: val_loss did not improve from 0.00995
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 84s 32ms/step - loss: 0.0115 - mae: 0.0115 - mape: 28.5910 - val_loss: 0.0114 - val_mae: 0.0114 - val_mape: 5.3488
Epoch 7/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0113 - mae: 0.0113 - mape: 459.3394
Epoch 7: val_loss did not improve from 0.00995
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 76s 29ms/step - loss: 0.0113 - mae: 0.0113 - mape: 459.2003 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.6029
Epoch 8/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0108 - mae: 0.0108 - mape: 11.9935
Epoch 8: val_loss did not improve from 0.00995
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 80s 28ms/step - loss: 0.0108 - mae: 0.0108 - mape: 12.0033 - val_loss: 0.0101 - val_mae

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 93s 35ms/step - loss: 0.0106 - mae: 0.0106 - mape: 151.0015 - val_loss: 0.0090 - val_mae: 0.0090 - val_mape: 4.3588
Epoch 10/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0103 - mae: 0.0103 - mape: 51.1950
Epoch 10: val_loss did not improve from 0.00901
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 116s 25ms/step - loss: 0.0103 - mae: 0.0103 - mape: 51.2000 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 4.3776


In [16]:

model = load_model(r'E:tasawar\ML_Labs\venv\Lab_9\\E1-cp-0002-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step
Mean Absolute Error (MAE): 181.86
Median Absolute Error (MedAE): 149.34
Mean Squared Error (MSE): 54589.97
Root Mean Squared Error (RMSE): 233.64
Mean Absolute Percentage Error (MAPE): 1.27 %
Median Absolute Percentage Error (MDAPE): 1.03 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [17]:
checkpoints = r'E:tasawar\ML_Labs\venv\Lab_9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ML_Labs\venv\Lab_9\E1-cp-0053-loss0.01.h5'
start_epoch= 54

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print("[INFO] loading model from disk...")
    model = load_model(r'E:\ML_Labs\venv\Lab_9\\E1-cp-0002-loss0.01.h5', compile=False)

    # Manually compile since we used compile=False
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

    # Print and update learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))
    model.optimizer.learning_rate = 1e-4

    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))


[INFO] loading model from disk...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [22]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0117 - mae: 0.0117 - mape: 39.1986
Epoch 1: val_loss did not improve from 0.00901
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 93s 30ms/step - loss: 0.0117 - mae: 0.0117 - mape: 39.1957 - val_loss: 0.0109 - val_mae: 0.0109 - val_mape: 4.9692
Epoch 2/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0114 - mae: 0.0114 - mape: 31.9093
Epoch 2: val_loss did not improve from 0.00901
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 91s 33ms/step - loss: 0.0114 - mae: 0.0114 - mape: 31.9710 - val_loss: 0.0109 - val_mae: 0.0109 - val_mape: 4.8930
Epoch 3/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0114 - mae: 0.0114 - mape: 20.2696
Epoch 3: val_loss did not improve from 0.00901
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 81s 31ms/step - loss: 0.0114 - mae: 0.0114 - mape: 20.2976 - val_loss: 0.0106 - val_mae: 0.0106 - val_mape: 4.5653
Epoch 4/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0111 - mae: 0.0111 - mape: 17.7643
Epoch 4: val_loss did

In [23]:

model = load_model(r'E:tasawar\ML_Labs\venv\Lab_9\\E1-cp-0002-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step
Mean Absolute Error (MAE): 181.86
Median Absolute Error (MedAE): 149.34
Mean Squared Error (MSE): 54589.97
Root Mean Squared Error (RMSE): 233.64
Mean Absolute Percentage Error (MAPE): 1.27 %
Median Absolute Percentage Error (MDAPE): 1.03 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
